In [6]:
!pip install git+https://github.com/monkeyhippies/xlnet.git

  Cloning https://github.com/monkeyhippies/xlnet.git to /tmp/pip-req-build-ufszd3oe
  Running command git clone -q https://github.com/monkeyhippies/xlnet.git /tmp/pip-req-build-ufszd3oe
  Stored in directory: /tmp/pip-ephem-wheel-cache-pqgqi6__/wheels/6e/9a/01/d0ca3ef8d5fc22f32dcdfc011fca0c53b65b4086a90eb71f9f
Successfully built xlnet


In [ ]:
!wget https://storage.googleapis.com/xlnet/released_models/cased_L-24_H-1024_A-16.zip
!unzip cased_L-24_H-1024_A-16.zip

--2019-06-25 22:36:30--  https://storage.googleapis.com/xlnet/released_models/cased_L-24_H-1024_A-16.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 108.177.127.128, 2a00:1450:4013:c07::80
Connecting to storage.googleapis.com (storage.googleapis.com)|108.177.127.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1338042341 (1.2G) [application/zip]
Saving to: ‘cased_L-24_H-1024_A-16.zip.1’

     cased_L-24_H-1  12%[=>                  ] 163.96M   164MB/s               

In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime
import tensorflow as tf
import tensorflow.contrib.slim as slim
from sklearn.model_selection import train_test_split

from xlnet.run_classifier import FLAGS
from xlnet.model_utils import get_train_op
from xlnet.run_classifier import InputExample
from xlnet.text_processor import TextProcessor
from xlnet.tf_record_utils import write_tf_record_file
from xlnet.input_fn_builders import tf_record_input_fn_builder
from xlnet import xlnet

# some code omitted here...
# initialize FLAGS
# Dummy initializer
LABELS_LIST = None
NUM_TRAIN_EPOCHS = 1
WARMUP_PROPORTION = .1
TEST_SIZE = .2
FLAGS(["python"])
FLAGS.model_config_path = "xlnet_cased_L-24_H-1024_A-16/xlnet_config.json"
FLAGS.spiece_model_file = "xlnet_cased_L-24_H-1024_A-16/spiece.model"
FLAGS.learning_rate = 2e-5
FLAGS.train_batch_size = 8
FLAGS.max_seq_length = 128
FLAGS.summary_type = "last"
FLAGS.use_summ_proj = True
# 1804874 is number of rows of original dataset
FLAGS.train_steps = int((1 - TEST_SIZE) * 1804874 / FLAGS.train_batch_size * NUM_TRAIN_EPOCHS)
FLAGS.warmup_steps = int(FLAGS.train_steps * WARMUP_PROPORTION)

SAVE_CHECKPOINTS_STEPS = 5000
SAVE_SUMMARY_STEPS = 1000
OUTPUT_DIR = "../"

In [ ]:
def write_tf_records(data, filename_suffix):
    """
    @data is dataframe
    """
    processor = TextProcessor(
        FLAGS.spiece_model_file,
        LABELS_LIST,
        FLAGS.max_seq_length,
        FLAGS.uncased
    )

    input_examples = [
        InputExample(
            guid=None,
            text_a=text,
            text_b=None,
            label=None
        )
        for text in data["comment_text"].values
    ]

    features = processor.process(input_examples)
    del input_examples

    write_tf_record_file(
        "toxicity-" + filename_suffix + ".tfrecord",
        features, data["target"].values
    )

In [ ]:
train = pd.read_csv("../input/train.csv")
train = train.iloc[: 10000]
train, val = train_test_split(train, shuffle=True,
    test_size=TEST_SIZE, random_state=42)

for filename_suffix, data in zip(["train", "val"], [train, val]):
    write_tf_records(data, filename_suffix)
    
del train, val, data

In [ ]:
train_input_fn = tf_record_input_fn_builder(
    ["toxicity-train.tfrecord"], FLAGS.max_seq_length, FLAGS.train_batch_size, 1000
)

val_input_fn = tf_record_input_fn_builder(
    ["toxicity-val.tfrecord"], FLAGS.max_seq_length, FLAGS.train_batch_size, 1000
)

In [ ]:
def create_model(flags, is_predicting, input_ids, input_mask, segment_ids, labels,
                 num_labels=None):
    """Creates a classification model."""

    xlnet_config = xlnet.XLNetConfig(json_path=FLAGS.model_config_path)

    # RunConfig contains hyperparameters that could be different between pretraining and finetuning.
    run_config = xlnet.create_run_config(
        is_training=True,
        is_finetune=True,
        FLAGS=flags
    )

    # Construct an XLNet model
    xlnet_model = xlnet.XLNetModel(
        xlnet_config=xlnet_config,
        run_config=run_config,
        input_ids=tf.transpose(input_ids),
        seg_ids=tf.transpose(segment_ids),
        input_mask=tf.transpose(input_mask)
    )

    # Get a summary of the sequence using the last hidden state
    output_layer = xlnet_model.get_pooled_out(
        summary_type=flags.summary_type,
        use_summ_proj=flags.use_summ_proj
    )

    with tf.variable_scope("loss"):
        output_layer = tf.nn.dropout(output_layer, keep_prob=0.9)
        logits = slim.fully_connected(
            output_layer,
            num_labels or 2,
            activation_fn=None,
            scope="fine_tune"
        )

        y_pred = tf.nn.softmax(logits)
        if num_labels is None:
            y_true = tf.concat(
                [tf.ones_like(labels) - labels, labels],
                -1
            )
        else:
            y_true = tf.one_hot(labels, depth=num_labels, dtype=tf.float32)
        
        predicted_labels = tf.squeeze(
            tf.argmax(
                y_pred, axis=-1, output_type=tf.int32
            )
        )

        # If we're predicting, we want predicted labels and the probabiltiies.
        if is_predicting:
            return (predicted_labels, y_pred)
        else:

            # If we're train/eval, compute loss between predicted and actual label
            #per_example_loss = -tf.reduce_sum(one_hot_labels * y_pred, axis=-1)
            #class_weights = tf.constant([[1.0, 50.0]])
            #weights = tf.reduce_sum(one_hot_labels * class_weights, axis=-1)
            loss = tf.nn.softmax_cross_entropy_with_logits_v2(
                y_true, logits
            )
            #loss = tf.reduce_mean(loss * weights)
            loss = tf.reduce_mean(loss)
            return (loss, predicted_labels, y_pred)

In [ ]:
# model_fn_builder actually creates our model function
# using the passed parameters for num_labels, learning_rate, etc.
def model_fn_builder(flags, num_labels=None):
    """Returns `model_fn` closure for TPUEstimator."""
    def model_fn(features, labels, mode, params):  # pylint: disable=unused-argument
        """The `model_fn` for TPUEstimator."""

        input_ids = features["input_ids"]
        input_mask = features["input_mask"]
        segment_ids = features["segment_ids"]
        #label_ids = features["label_ids"]

        is_predicting = (mode == tf.estimator.ModeKeys.PREDICT)

        # TRAIN and EVAL
        if not is_predicting:

            (loss, predicted_labels, log_probs) = create_model(
                flags, is_predicting, input_ids, input_mask, segment_ids,
                labels
            )

            train_op, _, _ = get_train_op(flags, loss)

            # Calculate evaluation metrics. 
            def metric_fn(label_ids, predicted_labels):
                accuracy = tf.metrics.accuracy(label_ids, predicted_labels)
                f1_score = tf.contrib.metrics.f1_score(
                    label_ids,
                    predicted_labels)
                auc = tf.metrics.auc(
                    label_ids,
                    predicted_labels)
                recall = tf.metrics.recall(
                    label_ids,
                    predicted_labels)
                precision = tf.metrics.precision(
                    label_ids,
                    predicted_labels) 
                true_pos = tf.metrics.true_positives(
                    label_ids,
                    predicted_labels)
                true_neg = tf.metrics.true_negatives(
                    label_ids,
                    predicted_labels)   
                false_pos = tf.metrics.false_positives(
                    label_ids,
                    predicted_labels)  
                false_neg = tf.metrics.false_negatives(
                    label_ids,
                    predicted_labels)
                return {
                    "eval_accuracy": accuracy,
                    "f1_score": f1_score,
                    "auc": auc,
                    "precision": precision,
                    "recall": recall,
                    "true_positives": true_pos,
                    "true_negatives": true_neg,
                    "false_positives": false_pos,
                    "false_negatives": false_neg
                }

            eval_metrics = metric_fn(labels, predicted_labels)

            if mode == tf.estimator.ModeKeys.TRAIN:
                return tf.estimator.EstimatorSpec(
                    mode=mode,
                    loss=loss,
                    train_op=train_op)
            else:
                return tf.estimator.EstimatorSpec(
                    mode=mode,
                    loss=loss,
                    eval_metric_ops=eval_metrics)
        else:
            (predicted_labels, log_probs) = create_model(
                flags, is_predicting, input_ids, input_mask,
                segment_ids, labels
            )

            predictions = {
                'probabilities': log_probs,
                'labels': predicted_labels
            }
            return tf.estimator.EstimatorSpec(mode, predictions=predictions)

    # Return the actual model function in the closure
    return model_fn


In [ ]:
# Specify outpit directory and number of checkpoint steps to save
run_config = tf.estimator.RunConfig(
    model_dir=OUTPUT_DIR,
    save_summary_steps=SAVE_SUMMARY_STEPS,
    save_checkpoints_steps=SAVE_CHECKPOINTS_STEPS
)
model_fn = model_fn_builder(FLAGS)

estimator = tf.estimator.Estimator(
  model_fn=model_fn,
  config=run_config,
  params={"batch_size": FLAGS.train_batch_size}
)

In [ ]:
for i in range(1, 11):
    steps = i * 5000
    print('Beginning Training! max_step: {}'.format(steps))
    current_time = datetime.now()
    estimator.train(input_fn=train_input_fn, max_steps=steps)
    print("Training took time ", datetime.now() - current_time)
    estimator.evaluate(input_fn=val_input_fn, steps=5)
